In [1]:
import pandas as pd
import numpy as np
from IPython.display import display, HTML
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_lookback_slices
from yf_utils import _5_perf_ranks, _6_grp_tuples_sort_sum
from myUtils import pickle_load, pickle_dump

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_colwidth', 16)
pd.set_option('display.width', 790)

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"
fp_df_close_clean = 'df_close_clean'
fp_df_eval_sym_freq_results = 'df_eval_sym_freq_results'
fp_df_eval_sym_freq_results_validate = 'df_eval_sym_freq_results_validate'
fp_df_eval_sym_freq_results_test = 'df_eval_sym_freq_results_test'

df_close_clean = pickle_load(path_data_dump, fp_df_close_clean)

In [2]:
col_add0 = ['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt']
col_add1 = ['grp(CAGR)_mean',         'grp(CAGR)_std',         'grp(CAGR)_mean/std']
col_add2 = ['grp(CAGR/UI)_mean',      'grp(CAGR/UI)_std',      'grp(CAGR/UI)_mean/std']
col_add3 = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
col_add4 = ['grp(retnStd/UI)_mean',   'grp(retnStd/UI)_std',   'grp(retnStd/UI)_mean/std']
col_add5 = ['SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI']

col_add_total = col_add0 + col_add1 + col_add2 + col_add3 + col_add4 + col_add5
print(f'col_add_total:\n{col_add_total}, total columns: {len(col_add_total)}')

col_add_total:
['n_samples', 'days_lookbacks', 'days_eval', 'n_top_syms', 'syms_start', 'syms_end', 'sym_freq_cnt', 'grp(CAGR)_mean', 'grp(CAGR)_std', 'grp(CAGR)_mean/std', 'grp(CAGR/UI)_mean', 'grp(CAGR/UI)_std', 'grp(CAGR/UI)_mean/std', 'grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std', 'grp(retnStd/UI)_mean', 'grp(retnStd/UI)_std', 'grp(retnStd/UI)_mean/std', 'SPY_CAGR', 'SPY_CAGR/UI', 'SPY_CAGR/retnStd', 'SPY_retnStd/UI'], total columns: 23


Warning: Create a new Dataframe to store run results. Previous data will be destroyed.

In [3]:
###################################  To create a new dataframe, select one choice from below
# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results)
# df

# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_validate)
# df

# # # Creating Empty DataFrame and save it to file
# df = pd.DataFrame(columns=col_add_total)
# pickle_dump(df, path_data_dump, fp_df_eval_sym_freq_results_test)
# df
###################################

Select type of run: Train, Validate or Test

In [4]:
###################################  Select type of run: train, validate or test
# df_eval_sym_freq_results = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
# # df_eval_sym_freq_results
# # Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# # The default split is 0.7, 0.2, 0.1 respectively.
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
# run_type = 'train'

# df_eval_sym_freq_results_validate = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_validate)
# # df_eval_sym_freq_results_validate
# # Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# # The default split is 0.7, 0.2, 0.1 respectively.
# df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
# df_train = df_test.copy()
# run_type = 'validate'

df_eval_sym_freq_results_test = pickle_load(path_data_dump, fp_df_eval_sym_freq_results_test)
# df_eval_sym_freq_results_test
# Split df_close_clean into training (df_train), validation (df_val) and test (df_test) set.
# The default split is 0.7, 0.2, 0.1 respectively.
df_train, df_val, df_test = _2_split_train_val_test(df_close_clean)
df_train = df_val.copy()
run_type = 'test'
###################################

In [5]:
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)
print(f'len_df_train: {len_df_train}, len_df_val: {len_df_val}, len_df_test: {len_df_test} ')

len_df_train: 300, len_df_val: 300, len_df_test: 150 


In [6]:
verbose = True  # True prints more output
# verbose = False  # True prints more output

# write run results to df_eval_results
# store_results = False
store_results = True

# number of max lookback tuples to create for iloc start_train:end_train:end_eval
# i.e. number of grp_top_set_syms_n_freq and grp_top_set_syms 
# n_samples = 400  
n_samples = 2

# for training, the number of days to lookback from iloc max-lookback end_train
days_lookbacks = [15]
days_lookbacks.sort()

# number of days from end_train are used to evaluate effectiveness of the training
days_eval = 6


# number of the most-common symbols from days_lookbacks' performance rankings to keep
n_top_syms = 20  

syms_start = 0  #  start index of n_top_syms for evaluation
syms_end = 10  #  end index of n_top_syms for evaluation

In [7]:
# Parameters
verbose = True
store_results = True
n_samples = 57
n_top_syms = 20
syms_start = 0
syms_end = 10
days_eval = 6
days_lookbacks = [15, 30, 60, 120]


create a sets of iloc lookback slices (start_train:end_train:end_eval), where  
end_train - start_train = days_lookback  
end_eval - end_train = days_eval  
for example,  
if given:  
 n_samples = 2  
 days_lookbacks = [30, 60, 120]  
 days_eval = 10  
a possible result is:  
 max_lookback_slices:  
 [(150, 270, 280), (5, 125, 135)]  
 where 270-150=125-5=max(days_lookbacks), 280-270=135-125=days_eval  
 sets_lookback_slices:  
 [[(240, 270, 280), (210, 270, 280), (150, 270, 280)], [(95, 125, 135), (65, 125, 135), (5, 125, 135)]]  
  where in a set, 270-240=days_lookbacks[0], 270-210=days_lookbacks[1], 270-150=days_lookbacks[2]  
  and 270, i.e. end_train, is constant for the set  

In [8]:
# return n_samples slices
max_lookback_slices = _3_random_slices(len_df_train, n_samples=n_samples, days_lookback=max(days_lookbacks), days_eval=days_eval)
# return n_samples * len(days_lookbacks) slices
sets_lookback_slices = _4_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=days_lookbacks, verbose=False)

if verbose:
  print(f'max_lookback_slices:\n{max_lookback_slices}')
  print(f'sets_lookback_slices:\n{sets_lookback_slices}')

max_lookback_slices:
[(86, 206, 212), (166, 286, 292), (160, 280, 286), (27, 147, 153), (103, 223, 229), (113, 233, 239), (15, 135, 141), (62, 182, 188), (1, 121, 127), (168, 288, 294), (102, 222, 228), (80, 200, 206), (14, 134, 140), (29, 149, 155), (129, 249, 255), (71, 191, 197), (68, 188, 194), (143, 263, 269), (154, 274, 280), (14, 134, 140), (60, 180, 186), (169, 289, 295), (102, 222, 228), (106, 226, 232), (58, 178, 184), (6, 126, 132), (54, 174, 180), (9, 129, 135), (38, 158, 164), (13, 133, 139), (79, 199, 205), (140, 260, 266), (51, 171, 177), (39, 159, 165), (44, 164, 170), (43, 163, 169), (16, 136, 142), (1, 121, 127), (36, 156, 162), (152, 272, 278), (69, 189, 195), (67, 187, 193), (121, 241, 247), (3, 123, 129), (17, 137, 143), (90, 210, 216), (133, 253, 259), (100, 220, 226), (26, 146, 152), (120, 240, 246), (150, 270, 276), (149, 269, 275), (45, 165, 171), (77, 197, 203), (170, 290, 296), (78, 198, 204), (86, 206, 212)]
sets_lookback_slices:
[[(191, 206, 212), (176, 206

Generate lists, n_samples long, of the highest performance ranked symbols. The performance metrics are: CAGR/UI, CAGR/retnStd, retnStd/UI. n_top_syms of the best performing symbols from each metric are combined. The symbols are sorted by their number of appearances in the combined pool, and are placed in a list. A slice of the best performing symbols is selected by syms_start:syms_end, i.e. top_set_syms_n_freq[syms_start:syms_end].     

The performance metrics are calculated based on slices in sets_lookback_slices.  The first two numbers are ilocs for training. The last two numbers are ilocs for evaluation.

In [9]:
grp_top_set_syms_n_freq = []  # list of lists of top_set_symbols_n_freq, there are n_samples lists in list
grp_top_set_syms = []  # list of lists of top_set_symbols, there are n_samples lists in list
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:
  grp_most_common_syms = []  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    d_eval = end_eval - start_eval

    _df = df_train.iloc[start_train:end_train]
    date_start_train = _df.index[0].strftime('%Y-%m-%d')
    date_end_train = _df.index[-1].strftime('%Y-%m-%d')

    if verbose:
      print(f'days lookback:    {lookback}')
      print(f'lb_slices:        {lb_slices}')
      print(f'lb_slice:         {lb_slice}')
      print(f'days eval:        {d_eval}')    
      print(f'iloc_start_train: {start_train}')
      print(f'iloc_end_train:   {end_train}')
      print(f'date_start_train: {date_start_train}')
      print(f'date_end_train:   {date_end_train}')


    perf_ranks, most_common_syms = _5_perf_ranks(_df, n_top_syms=n_top_syms)
    grp_most_common_syms.append(most_common_syms)
    
    if verbose:    
      # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
      print(f'perf_ranks: {perf_ranks}')  
      # most common symbols of perf_ranks 
      print(f'most_common_syms: {most_common_syms}')     
      # grp_perf_ranks[lookback] = perf_ranks
      print(f'+++ finish lookback slice {lookback} +++\n')

  if verbose:
    print(f'grp_most_common_syms: {grp_most_common_syms}')
    # grp_most_common_syms a is list of lists of tuples of 
    #  the most-common-symbols symbol:frequency cumulated from
    #  each days_lookback  
    print(f'**** finish lookback slices {lb_slices} ****\n')

  # flatten list of lists of (symbol:frequency)
  flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]
  # group symbols from set of days_lookbacks (i.e. lb_slices) and sum frequency of the symbols
  set_most_common_syms = _6_grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
  # get the top few most-frequent symbol:frequency pairs
  top_set_syms_n_freq = set_most_common_syms[0:n_top_syms]
  # get symbols from top_set_syms_n_freq

###################################  
  # top_set_syms = [i[0] for i in top_set_syms_n_freq]
  top_set_syms = [i[0] for i in top_set_syms_n_freq[syms_start:syms_end]]  
###################################  
  
  grp_top_set_syms_n_freq.append(top_set_syms_n_freq)
  grp_top_set_syms.append(top_set_syms)

  if verbose:  
    print(f'top {n_top_syms} ranked symbols and frequency from set {lb_slices}:\n{top_set_syms_n_freq}')
    print(f'top {n_top_syms} ranked symbols from set {lb_slices}:\n{top_set_syms}')  
    print(f'===== finish top {n_top_syms} ranked symbols from days_lookback set {lb_slices} =====\n\n')

days lookback:    15
lb_slices:        [(191, 206, 212), (176, 206, 212), (146, 206, 212), (86, 206, 212)]
lb_slice:         (191, 206, 212)
days eval:        6
iloc_start_train: 191
iloc_end_train:   206
date_start_train: 2022-02-16
date_end_train:   2022-03-09


perf_ranks: {'period-15': {'r_CAGR/UI': array(['LNTH', 'IPI', 'OXY', 'KAR', 'RRC', 'GTLS', 'AR', 'CVX', 'ROCC',
       'FLR', 'SSRM', 'GOGO', 'X', 'BEP', 'FHN', 'ACRS', 'RGLD', 'MRCY',
       'GLNG', 'MOS'], dtype=object), 'r_CAGR/retnStd': array(['LNTH', 'IPI', 'OXY', 'KAR', 'GTLS', 'RRC', 'AR', 'ROCC', 'FLR',
       'CVX', 'DQ', 'X', 'CCJ', 'MOS', 'MRCY', 'ACRS', 'BEP', 'KR',
       'GOGO', 'GLNG'], dtype=object), 'r_retnStd/UI': array(['LNTH', 'SSRM', 'SRE', 'OXY', 'RRC', 'FHN', 'RGLD', 'FDUS', 'IPI',
       'CVX', 'KAR', 'NEM', 'ROCC', 'GOGO', 'FPI', 'FLR', 'EPR', 'AR',
       'CPE', 'VTR'], dtype=object)}}
most_common_syms: [('LNTH', 3), ('IPI', 3), ('OXY', 3), ('KAR', 3), ('RRC', 3), ('AR', 3), ('CVX', 3), ('ROCC', 3), ('FLR', 3), ('GOGO', 3), ('GTLS', 2), ('SSRM', 2), ('X', 2), ('BEP', 2), ('FHN', 2), ('ACRS', 2), ('RGLD', 2), ('MRCY', 2), ('GLNG', 2), ('MOS', 2), ('DQ', 1), ('CCJ', 1), ('KR', 1), ('SRE', 1), ('FDUS', 1), ('NEM', 1), ('FPI', 1), ('EPR', 1), ('CPE', 1), ('VTR', 1

perf_ranks: {'period-30': {'r_CAGR/UI': array(['LNTH', 'IPI', 'X', 'CORT', 'PTEN', 'ACRS', 'SBLK', 'HP', 'ARCH',
       'STNG', 'MOS', 'OXY', 'RRC', 'EURN', 'CPRX', 'CLF', 'NUE', 'ASC',
       'RGLD', 'TSEM'], dtype=object), 'r_CAGR/retnStd': array(['IPI', 'LNTH', 'X', 'PTEN', 'HP', 'SBLK', 'ACRS', 'OXY', 'ARCH',
       'CLF', 'CORT', 'STNG', 'MOS', 'RRC', 'AR', 'GTLS', 'NEX', 'NUE',
       'SQM', 'BPT'], dtype=object), 'r_retnStd/UI': array(['LNTH', 'TSEM', 'FHN', 'CORT', 'IPI', 'FDUS', 'RGLD', 'ATI',
       'SSRM', 'EURN', 'NEM', 'GOLD', 'NJR', 'STNG', 'ACRS', 'IRT', 'X',
       'WPM', 'ARCH', 'HSIC'], dtype=object)}}
most_common_syms: [('LNTH', 3), ('IPI', 3), ('X', 3), ('CORT', 3), ('ACRS', 3), ('ARCH', 3), ('STNG', 3), ('PTEN', 2), ('SBLK', 2), ('HP', 2), ('MOS', 2), ('OXY', 2), ('RRC', 2), ('EURN', 2), ('CLF', 2), ('NUE', 2), ('RGLD', 2), ('TSEM', 2), ('CPRX', 1), ('ASC', 1), ('AR', 1), ('GTLS', 1), ('NEX', 1), ('SQM', 1), ('BPT', 1), ('FHN', 1), ('FDUS', 1), ('ATI', 1), ('SSRM',

perf_ranks: {'period-60': {'r_CAGR/UI': array(['BPT', 'OXY', 'PTEN', 'NEX', 'ARCH', 'ATI', 'HP', 'CVX', 'HAL',
       'GLNG', 'MRO', 'USO', 'PBF', 'AA', 'IPI', 'NOV', 'LNTH', 'MOS',
       'CNQ', 'LMT'], dtype=object), 'r_CAGR/retnStd': array(['BPT', 'NEX', 'OXY', 'HP', 'PTEN', 'IPI', 'ATI', 'PBF', 'ARCH',
       'LNTH', 'GLNG', 'CVX', 'MOS', 'AA', 'HAL', 'MRO', 'NOV', 'CNQ',
       'TECK', 'ARLP'], dtype=object), 'r_retnStd/UI': array(['USO', 'CVX', 'LMT', 'PTEN', 'TRP', 'DVN', 'HAL', 'MRO', 'ARCH',
       'FHN', 'IMO', 'AMPH', 'OXY', 'ADM', 'CVE', 'GLNG', 'DCP', 'MO',
       'NOV', 'ATI'], dtype=object)}}
most_common_syms: [('OXY', 3), ('PTEN', 3), ('ARCH', 3), ('ATI', 3), ('CVX', 3), ('HAL', 3), ('GLNG', 3), ('MRO', 3), ('NOV', 3), ('BPT', 2), ('NEX', 2), ('HP', 2), ('USO', 2), ('PBF', 2), ('AA', 2), ('IPI', 2), ('LNTH', 2), ('MOS', 2), ('CNQ', 2), ('LMT', 2), ('TECK', 1), ('ARLP', 1), ('TRP', 1), ('DVN', 1), ('FHN', 1), ('IMO', 1), ('AMPH', 1), ('ADM', 1), ('CVE', 1), ('DCP', 1), (

perf_ranks: {'period-15': {'r_CAGR/UI': array(['PTCT', 'AXSM', 'NVAX', 'NTLA', 'FATE', 'QURE', 'AVXL', 'BGNE',
       'CYRX', 'FOLD', 'RCKT', 'ARWR', 'RARE', 'ELF', 'SGEN', 'CELH',
       'RGNX', 'AGYS', 'FDP', 'WIX'], dtype=object), 'r_CAGR/retnStd': array(['AXSM', 'NVAX', 'PTCT', 'FATE', 'NTLA', 'QURE', 'BGNE', 'AVXL',
       'CYRX', 'RCKT', 'FOLD', 'ARWR', 'RGNX', 'RARE', 'ELF', 'CELH',
       'TDOC', 'AGYS', 'FDP', 'WIX'], dtype=object), 'r_retnStd/UI': array(['PTCT', 'AXSM', 'SGEN', 'NTLA', 'LRN', 'GIS', 'QURE', 'ABBV',
       'MODN', 'SJR', 'O', 'EBF', 'ELF', 'RARE', 'VNDA', 'SIGI', 'AMPH',
       'HUM', 'CAG', 'DG'], dtype=object)}}
most_common_syms: [('PTCT', 3), ('AXSM', 3), ('NTLA', 3), ('QURE', 3), ('RARE', 3), ('ELF', 3), ('NVAX', 2), ('FATE', 2), ('AVXL', 2), ('BGNE', 2), ('CYRX', 2), ('FOLD', 2), ('RCKT', 2), ('ARWR', 2), ('SGEN', 2), ('CELH', 2), ('RGNX', 2), ('AGYS', 2), ('FDP', 2), ('WIX', 2), ('TDOC', 1), ('LRN', 1), ('GIS', 1), ('ABBV', 1), ('MODN', 1), ('SJR', 1), (

...

perf_ranks: {'period-120': {'r_CAGR/UI': array(['BPT', 'DVN', 'CVX', 'ERF', 'IRTC', 'MRO', 'CVE', 'FANG', 'ABBV',
       'COP', 'PXD', 'IMO', 'EOG', 'HUN', 'CNQ', 'VIRT', 'CLMT', 'AMPH',
       'SU', 'MCK'], dtype=object), 'r_CAGR/retnStd': array(['BPT', 'MRO', 'ERF', 'DVN', 'IRTC', 'EOG', 'CVE', 'COP', 'CVX',
       'ABBV', 'CLMT', 'IMO', 'PXD', 'FANG', 'HUN', 'CNQ', 'SU', 'HAL',
       'XOM', 'NEX'], dtype=object), 'r_retnStd/UI': array(['CVX', 'DVN', 'AMPH', 'BCC', 'ERF', 'VIRT', 'XENE', 'TD', 'TSEM',
       'IRTC', 'FANG', 'GIS', 'MCK', 'TSN', 'MRO', 'ABBV', 'CME', 'CNQ',
       'CVE', 'HUN'], dtype=object)}}
most_common_syms: [('DVN', 3), ('CVX', 3), ('ERF', 3), ('IRTC', 3), ('MRO', 3), ('CVE', 3), ('FANG', 3), ('ABBV', 3), ('HUN', 3), ('CNQ', 3), ('BPT', 2), ('COP', 2), ('PXD', 2), ('IMO', 2), ('EOG', 2), ('VIRT', 2), ('CLMT', 2), ('AMPH', 2), ('SU', 2), ('MCK', 2), ('HAL', 1), ('XOM', 1), ('NEX', 1), ('BCC', 1), ('XENE', 1), ('TD', 1), ('TSEM', 1), ('GIS', 1), ('TSN', 1), ('CME'

perf_ranks: {'period-60': {'r_CAGR/UI': array(['EDU', 'STNG', 'DQ', 'STKL', 'HRB', 'RFP', 'AMBC', 'MNST', 'DORM',
       'CELH', 'CLW', 'THS', 'ASC', 'BPT', 'AGYS', 'FCN', 'PRGO', 'IBM',
       'FOLD', 'WAT'], dtype=object), 'r_CAGR/retnStd': array(['EDU', 'DQ', 'HRB', 'STNG', 'STKL', 'AMBC', 'RFP', 'CELH', 'FCN',
       'PRGO', 'AGYS', 'DORM', 'BPT', 'FOLD', 'HALO', 'MNST', 'ASC',
       'CLW', 'WAT', 'THS'], dtype=object), 'r_retnStd/UI': array(['MNST', 'RFP', 'STKL', 'STNG', 'FDX', 'THS', 'HRB', 'OHI', 'DORM',
       'EDU', 'CLW', 'DQ', 'SBRA', 'ASC', 'CYTK', 'KDNY', 'OSIS', 'PBH',
       'CELH', 'IBM'], dtype=object)}}
most_common_syms: [('EDU', 3), ('STNG', 3), ('DQ', 3), ('STKL', 3), ('HRB', 3), ('RFP', 3), ('MNST', 3), ('DORM', 3), ('CELH', 3), ('CLW', 3), ('THS', 3), ('ASC', 3), ('AMBC', 2), ('BPT', 2), ('AGYS', 2), ('FCN', 2), ('PRGO', 2), ('IBM', 2), ('FOLD', 2), ('WAT', 2), ('HALO', 1), ('FDX', 1), ('OHI', 1), ('SBRA', 1), ('CYTK', 1), ('KDNY', 1), ('OSIS', 1), ('PBH', 1)]
+

perf_ranks: {'period-30': {'r_CAGR/UI': array(['BPT', 'LNTH', 'SBLK', 'ACRS', 'PTEN', 'STNG', 'SBS', 'ITCI',
       'MGI', 'AA', 'NJR', 'VIRT', 'ASC', 'ATI', 'RGLD', 'GLNG', 'TGNA',
       'TIMB', 'USO', 'ABBV'], dtype=object), 'r_CAGR/retnStd': array(['BPT', 'LNTH', 'SBLK', 'ACRS', 'PTEN', 'STNG', 'ITCI', 'SBS',
       'MGI', 'AA', 'ASC', 'GLNG', 'ATI', 'VIRT', 'CPRX', 'ABBV', 'MATX',
       'HTHT', 'IPI', 'NEX'], dtype=object), 'r_retnStd/UI': array(['LNTH', 'NJR', 'TRV', 'RGLD', 'TIMB', 'PTEN', 'LVS', 'PBR', 'STNG',
       'VIRT', 'SBS', 'CAG', 'AA', 'AMPH', 'TGNA', 'CVX', 'BCC', 'MGI',
       'VOD', 'DVN'], dtype=object)}}
most_common_syms: [('LNTH', 3), ('PTEN', 3), ('STNG', 3), ('SBS', 3), ('MGI', 3), ('AA', 3), ('VIRT', 3), ('BPT', 2), ('SBLK', 2), ('ACRS', 2), ('ITCI', 2), ('NJR', 2), ('ASC', 2), ('ATI', 2), ('RGLD', 2), ('GLNG', 2), ('TGNA', 2), ('TIMB', 2), ('ABBV', 2), ('USO', 1), ('CPRX', 1), ('MATX', 1), ('HTHT', 1), ('IPI', 1), ('NEX', 1), ('TRV', 1), ('LVS', 1), ('PBR', 

perf_ranks: {'period-15': {'r_CAGR/UI': array(['LNTH', 'IPI', 'OXY', 'KAR', 'RRC', 'GTLS', 'AR', 'CVX', 'ROCC',
       'FLR', 'SSRM', 'GOGO', 'X', 'BEP', 'FHN', 'ACRS', 'RGLD', 'MRCY',
       'GLNG', 'MOS'], dtype=object), 'r_CAGR/retnStd': array(['LNTH', 'IPI', 'OXY', 'KAR', 'GTLS', 'RRC', 'AR', 'ROCC', 'FLR',
       'CVX', 'DQ', 'X', 'CCJ', 'MOS', 'MRCY', 'ACRS', 'BEP', 'KR',
       'GOGO', 'GLNG'], dtype=object), 'r_retnStd/UI': array(['LNTH', 'SSRM', 'SRE', 'OXY', 'RRC', 'FHN', 'RGLD', 'FDUS', 'IPI',
       'CVX', 'KAR', 'NEM', 'ROCC', 'GOGO', 'FPI', 'FLR', 'EPR', 'AR',
       'CPE', 'VTR'], dtype=object)}}
most_common_syms: [('LNTH', 3), ('IPI', 3), ('OXY', 3), ('KAR', 3), ('RRC', 3), ('AR', 3), ('CVX', 3), ('ROCC', 3), ('FLR', 3), ('GOGO', 3), ('GTLS', 2), ('SSRM', 2), ('X', 2), ('BEP', 2), ('FHN', 2), ('ACRS', 2), ('RGLD', 2), ('MRCY', 2), ('GLNG', 2), ('MOS', 2), ('DQ', 1), ('CCJ', 1), ('KR', 1), ('SRE', 1), ('FDUS', 1), ('NEM', 1), ('FPI', 1), ('EPR', 1), ('CPE', 1), ('VTR', 1

perf_ranks: {'period-120': {'r_CAGR/UI': array(['CVX', 'ABBV', 'DVN', 'BPT', 'ERF', 'MRO', 'AMPH', 'CNQ', 'CVE',
       'IMO', 'COP', 'IPI', 'OXY', 'SU', 'MDRX', 'AU', 'MCK', 'XOM',
       'PXD', 'ATO'], dtype=object), 'r_CAGR/retnStd': array(['BPT', 'CVX', 'IPI', 'ERF', 'MRO', 'DVN', 'OXY', 'CNQ', 'ABBV',
       'CVE', 'IMO', 'COP', 'PBF', 'APA', 'PTEN', 'HAL', 'TRGP', 'XOM',
       'ARCH', 'LNTH'], dtype=object), 'r_retnStd/UI': array(['CVX', 'ABBV', 'AMPH', 'DVN', 'FHN', 'BCC', 'ATO', 'ERF', 'FE',
       'NI', 'MCK', 'VIRT', 'XENE', 'MDRX', 'FMC', 'TSEM', 'ED', 'FANG',
       'VST', 'BNS'], dtype=object)}}
most_common_syms: [('CVX', 3), ('ABBV', 3), ('DVN', 3), ('ERF', 3), ('BPT', 2), ('MRO', 2), ('AMPH', 2), ('CNQ', 2), ('CVE', 2), ('IMO', 2), ('COP', 2), ('IPI', 2), ('OXY', 2), ('MDRX', 2), ('MCK', 2), ('XOM', 2), ('ATO', 2), ('SU', 1), ('AU', 1), ('PXD', 1), ('PBF', 1), ('APA', 1), ('PTEN', 1), ('HAL', 1), ('TRGP', 1), ('ARCH', 1), ('LNTH', 1), ('FHN', 1), ('BCC', 1), ('FE', 1), 

In [10]:
def top_set_sym_freq_cnt(top_set_syms_n_freq):
    # accommodate upto 5 periods of days_lookbacks(i.e. days_lookbacks = [5, 10, 15, 20, 25])
    sym_freq_cnt_15 = []
    sym_freq_cnt_14 = []
    sym_freq_cnt_13 = []
    sym_freq_cnt_12 = []
    sym_freq_cnt_11 = []
    sym_freq_cnt_10 = []
    sym_freq_cnt_9 = []
    sym_freq_cnt_8 = []
    sym_freq_cnt_7 = []
    sym_freq_cnt_6 = []
    sym_freq_cnt_5 = []
    sym_freq_cnt_4 = []
    sym_freq_cnt_3 = []
    sym_freq_cnt_2 = []

    for sym_n_freq in top_set_syms_n_freq:
        _sym = sym_n_freq[0]
        _freq = sym_n_freq[1]
        # print(_sym, _freq)
        if _freq == 15:
            sym_freq_cnt_15.append(_sym)
        elif _freq == 14:
            sym_freq_cnt_14.append(_sym)
        elif _freq == 13:
            sym_freq_cnt_13.append(_sym)
        elif _freq == 12:
            sym_freq_cnt_12.append(_sym)                        
        elif _freq == 11:
            sym_freq_cnt_11.append(_sym)
        elif _freq == 10:
            sym_freq_cnt_10.append(_sym)            
        elif _freq == 9:
            sym_freq_cnt_9.append(_sym)
        elif _freq == 8:
            sym_freq_cnt_8.append(_sym)
        elif _freq == 7:
            sym_freq_cnt_7.append(_sym)  
        elif _freq == 6:
            sym_freq_cnt_6.append(_sym)
        elif _freq == 5:
            sym_freq_cnt_5.append(_sym)
        elif _freq == 4:
            sym_freq_cnt_4.append(_sym)
        elif _freq == 3:
            sym_freq_cnt_3.append(_sym)          
        else:
            sym_freq_cnt_2.append(_sym)

    l_sym_freq_cnt = []

    l_sym_freq_cnt.append(sym_freq_cnt_15)
    l_sym_freq_cnt.append(sym_freq_cnt_14)
    l_sym_freq_cnt.append(sym_freq_cnt_13)
    l_sym_freq_cnt.append(sym_freq_cnt_12)    
    l_sym_freq_cnt.append(sym_freq_cnt_11)   
    l_sym_freq_cnt.append(sym_freq_cnt_10)
    l_sym_freq_cnt.append(sym_freq_cnt_9)
    l_sym_freq_cnt.append(sym_freq_cnt_8)
    l_sym_freq_cnt.append(sym_freq_cnt_7)    
    l_sym_freq_cnt.append(sym_freq_cnt_6)
    l_sym_freq_cnt.append(sym_freq_cnt_5)
    l_sym_freq_cnt.append(sym_freq_cnt_4)
    l_sym_freq_cnt.append(sym_freq_cnt_3)    
    l_sym_freq_cnt.append(sym_freq_cnt_2)    

    return l_sym_freq_cnt    

In [11]:
for top_set_syms_n_freq in grp_top_set_syms_n_freq:
  l_sym_freq_cnt = top_set_sym_freq_cnt(top_set_syms_n_freq)
  if verbose:
    print(f'sym_freq_15: {l_sym_freq_cnt[0]}')
    print(f'sym_freq_14: {l_sym_freq_cnt[1]}')
    print(f'sym_freq_13: {l_sym_freq_cnt[2]}')
    print(f'sym_freq_12: {l_sym_freq_cnt[3]}')
    print(f'sym_freq_11: {l_sym_freq_cnt[4]}')
    print(f'sym_freq_10: {l_sym_freq_cnt[5]}')
    print(f'sym_freq_9: {l_sym_freq_cnt[6]}')
    print(f'sym_freq_8: {l_sym_freq_cnt[7]}')
    print(f'sym_freq_7: {l_sym_freq_cnt[8]}')
    print(f'sym_freq_6: {l_sym_freq_cnt[9]}')
    print(f'sym_freq_5: {l_sym_freq_cnt[10]}')
    print(f'sym_freq_4: {l_sym_freq_cnt[11]}')
    print(f'sym_freq_3: {l_sym_freq_cnt[12]}')
    print(f'sym_freq_2: {l_sym_freq_cnt[13]}\n')  

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: ['IPI', 'OXY']
sym_freq_9: ['CVX', 'LNTH']
sym_freq_8: []
sym_freq_7: ['ARCH']
sym_freq_6: ['MOS', 'PTEN']
sym_freq_5: ['ACRS', 'BPT', 'FHN', 'GLNG', 'MRO', 'RRC', 'X']
sym_freq_4: ['AR', 'ATI', 'CNQ', 'DVN', 'HAL', 'HP']
sym_freq_3: []
sym_freq_2: []

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: []
sym_freq_8: ['DQ', 'ELF']
sym_freq_7: []
sym_freq_6: ['ASC', 'HRB', 'QURE', 'STNG']
sym_freq_5: ['CELH', 'CYRX', 'EDU', 'FDP', 'FOLD', 'RARE']
sym_freq_4: ['AGYS', 'ATHM', 'BGNE', 'CVBF', 'DG', 'FCN', 'PTCT', 'RCKT']
sym_freq_3: []
sym_freq_2: []

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: ['DQ', 'EDU', 'STNG']
sym_freq_8: []
sym_freq_7: []
sym_freq_6: ['ASC', 'BPT', 'FOLD', 'HRB', 'SGEN']
sym_freq_5: ['QURE']
sym_freq_4: ['ARCH', 'BABA', 'CORT', 'DPZ', 'RAMP']
sym_freq_3: 

In [12]:
# zip_cnt_n_syms = zip([15,14,13,12,11,10,9,8,7,6,5,4,3], l_sym_freq_cnt[:-1])
# for _cnt, _syms in zip_cnt_n_syms:
#   print(_cnt, _syms)    

In [13]:
from myUtils import symb_perf_stats_vectorized_v8

print('z_grp_top_set_syms_n_freq:')
z_grp_top_set_syms_n_freq = zip(max_lookback_slices, grp_top_set_syms_n_freq)

for i, (_lookback_slice, _top_set_syms_n_freq) in enumerate(z_grp_top_set_syms_n_freq):
  start_train = _lookback_slice[0]
  end_train = _lookback_slice[1]
  start_eval = end_train
  end_eval = _lookback_slice[2]

  print(f'{i + 1 } of {n_samples} max_lookback_slice')
  print(f'max_lookback_slice: {_lookback_slice}')
  # dates correspond to max_lookback_slice
  d_start_train = df_train.index[start_train].strftime('%Y-%m-%d')
  d_end_train = df_train.index[end_train].strftime('%Y-%m-%d')
  d_start_eval = d_end_train
  d_end_eval = df_train.index[end_eval].strftime('%Y-%m-%d')
  d_df_eval_start = d_end_train
  d_df_eval_end = df_train.index[end_eval - 1].strftime('%Y-%m-%d')  
  print(f'max lookback dates: {d_start_train}, {d_end_train}, {d_end_eval}')
  print(f'df_eval dates (inclusive): {d_df_eval_start} - {d_df_eval_end}')    
  print(f'top_set_syms_n_freq: {_top_set_syms_n_freq}\n')

  l_sym_freq_cnt = top_set_sym_freq_cnt(_top_set_syms_n_freq)
  if verbose:
    print(f'sym_freq_15: {l_sym_freq_cnt[0]}')
    print(f'sym_freq_14: {l_sym_freq_cnt[1]}')
    print(f'sym_freq_13: {l_sym_freq_cnt[2]}')
    print(f'sym_freq_12: {l_sym_freq_cnt[3]}')
    print(f'sym_freq_11: {l_sym_freq_cnt[4]}')
    print(f'sym_freq_10: {l_sym_freq_cnt[5]}')
    print(f'sym_freq_9: {l_sym_freq_cnt[6]}')
    print(f'sym_freq_8: {l_sym_freq_cnt[7]}')
    print(f'sym_freq_7: {l_sym_freq_cnt[8]}')
    print(f'sym_freq_6: {l_sym_freq_cnt[9]}')
    print(f'sym_freq_5: {l_sym_freq_cnt[10]}')
    print(f'sym_freq_4: {l_sym_freq_cnt[11]}')
    print(f'sym_freq_3: {l_sym_freq_cnt[12]}')
    print(f'sym_freq_2: {l_sym_freq_cnt[13]}\n')  


  _sym_idx = ['SPY']
  df_SPY = df_train[start_eval:end_eval][_sym_idx]

  (
    _symbols,
    _period_yr,
    _retn,
    _DD,
    _UI,
    _MDD,
    _retnMean,
    _retnStd,
    _retnStd_div_UI,
    _CAGR,
    _CAGR_div_retnStd,
    _CAGR_div_UI,
    SPY_retnStd_d_UI,     
    SPY_CAGR,
    SPY_CAGR_d_retnStd,
    SPY_CAGR_d_UI,
  ) = symb_perf_stats_vectorized_v8(df_SPY)  

  print(f'\nSPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR: {SPY_retnStd_d_UI[0]:>17,.1f}, {SPY_CAGR_d_retnStd[0]:>17,.1f}, {SPY_CAGR_d_UI[0]:>17.1f}, {SPY_CAGR[0]:>17.1f}')

  # drop last list, with frequency count 2 or less, in l_sym_freq_cnt from zip
  # zip_cnt_n_syms = zip([6,5,4,3], l_sym_freq_cnt[:-1])  
  zip_cnt_n_syms = zip([15,14,13,12,11,10,9,8,7,6,5,4,3], l_sym_freq_cnt[:-1])
  for item in zip_cnt_n_syms:
    sym_freq_cnt = item[0]
    syms = item[1]
    if syms:  # iterate ONLY if there are symbols in syms
      df_eval = df_train[start_eval:end_eval][syms]      

      if verbose:
        print(f'start_eval: {start_eval},  date: {d_end_train}')
        print(f'end_eval:   {end_eval},  date: {d_end_eval},  df_eval last date: {d_df_eval_end}')
        print(f'frequency count of symbol(s): {sym_freq_cnt}')      
        # print(f'\ndf_eval:\n{df_eval}\n')

        syms_n_SPY = syms + ['SPY']
        df_eval_n_SPY = df_train[start_eval:end_eval][syms_n_SPY]   
        print(f'\ndf_eval_n_SPY:\n{df_eval_n_SPY}\n')

      (
        _symbols,
        _period_yr,
        _retn,
        _DD,
        _UI,
        _MDD,
        _retnMean,
        _retnStd,
        _retnStd_div_UI,
        _CAGR,
        _CAGR_div_retnStd,
        _CAGR_div_UI,
        grp_retnStd_d_UI,     
        grp_CAGR,
        grp_CAGR_d_retnStd,
        grp_CAGR_d_UI,
      ) = symb_perf_stats_vectorized_v8(df_eval)  
      print(f'grp(retnStd/UI):   mean, std, mean/std: {grp_retnStd_d_UI[0]  :>17,.1f}, {grp_retnStd_d_UI[1]  :>17,.1f}, {grp_retnStd_d_UI[2]  :>17,.1f}')
      print(f'grp(CAGR/retnStd): mean, std, mean/std: {grp_CAGR_d_retnStd[0]:>17,.1f}, {grp_CAGR_d_retnStd[1]:>17,.1f}, {grp_CAGR_d_retnStd[2]:>17,.1f}')
      print(f'grp(CAGR/UI):      mean, std, mean/std: {grp_CAGR_d_UI[0]     :>17,.1f}, {grp_CAGR_d_UI[1]     :>17,.1f}, {grp_CAGR_d_UI[2]     :>17,.1f}')
      print(f'grp(CAGR):         mean, std, mean/std: {grp_CAGR[0]          :>17,.1f}, {grp_CAGR[1]          :>17,.1f}, {grp_CAGR[2]          :>17,.1f}')

      if store_results:  # record results to df
        row_add0      = [n_samples, str(days_lookbacks), days_eval, n_top_syms, syms_start, syms_end, sym_freq_cnt]
        row_add1      = [grp_CAGR[0],           grp_CAGR[1],           grp_CAGR[2]]
        row_add2      = [grp_CAGR_d_UI[0],      grp_CAGR_d_UI[1],      grp_CAGR_d_UI[2]]
        row_add3      = [grp_CAGR_d_retnStd[0], grp_CAGR_d_retnStd[1], grp_CAGR_d_retnStd[2]]
        row_add4      = [grp_retnStd_d_UI[0],   grp_retnStd_d_UI[1],   grp_retnStd_d_UI[2]]
        row_add5      = [SPY_CAGR[0], SPY_CAGR_d_UI[0], SPY_CAGR_d_retnStd[0], SPY_retnStd_d_UI[0]]
        row_add_total = row_add0 + row_add1 + row_add2 + row_add3 + row_add4 + row_add5
        print(f'row_add_total: {row_add_total}')

        if run_type == 'train':
          df_eval_sym_freq_results.loc[len(df_eval_sym_freq_results)] = row_add_total
          print(f'appended row_add to df_eval_sym_freq_results:\n{row_add_total}\n')
        elif run_type == 'validate':
          df_eval_sym_freq_results_validate.loc[len(df_eval_sym_freq_results_validate)] = row_add_total       
          print(f'appended row_add to df_eval_sym_freq_results_validate:\n{row_add_total}\n')
        elif run_type == 'test':
          df_eval_sym_freq_results_test.loc[len(df_eval_sym_freq_results_test)] = row_add_total       
          print(f'appended row_add to df_eval_sym_freq_results_test:\n{row_add_total}\n')
        else:
          msg_stop = f"ERROR run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
          raise SystemExit(msg_stop )                             
  
  print('='*50, '\n')  

z_grp_top_set_syms_n_freq:
1 of 57 max_lookback_slice
max_lookback_slice: (86, 206, 212)
max lookback dates: 2021-09-17, 2022-03-10, 2022-03-18
df_eval dates (inclusive): 2022-03-10 - 2022-03-17
top_set_syms_n_freq: [('IPI', 10), ('OXY', 10), ('CVX', 9), ('LNTH', 9), ('ARCH', 7), ('MOS', 6), ('PTEN', 6), ('ACRS', 5), ('BPT', 5), ('FHN', 5), ('GLNG', 5), ('MRO', 5), ('RRC', 5), ('X', 5), ('AR', 4), ('ATI', 4), ('CNQ', 4), ('DVN', 4), ('HAL', 4), ('HP', 4)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: ['IPI', 'OXY']
sym_freq_9: ['CVX', 'LNTH']
sym_freq_8: []
sym_freq_7: ['ARCH']
sym_freq_6: ['MOS', 'PTEN']
sym_freq_5: ['ACRS', 'BPT', 'FHN', 'GLNG', 'MRO', 'RRC', 'X']
sym_freq_4: ['AR', 'ATI', 'CNQ', 'DVN', 'HAL', 'HP']
sym_freq_3: []
sym_freq_2: []


SPY: retnStd/UI, CAGR/retnStd, CAGR/UI, CAGR:               1.5,             349.5,             531.8,               5.1
start_eval: 206,  date: 2022-03-10
end_eval:   212,  date: 2022-03-18, 

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 8, 2.2719757896262087, 3.064738050079264, 0.7413278892032708, 14.255056557743867, 47.830524638260215, 0.2980326196619026, -36.961463015330224, 101.93716779775343, -0.36259064101783695, 0.5986231749105211, 0.356896759869764, 1.6773006712892715, -0.44269315239041196, -30.396351925042893, -47.43337711095151, 0.6408220071268113]

start_eval: 286,  date: 2022-07-06
end_eval:   292,  date: 2022-07-14,  df_eval last date: 2022-07-13
frequency count of symbol(s): 6

df_eval_n_SPY:
                 ASC        HRB       QURE       STNG         SPY
Date                                                             
2022-07-06  6.321090  36.260189  20.190001  29.162584  379.937439
2022-07-07  6.682016  35.964184  21.910000  32.079842  385.627808
2022-07-08  6.769809  36.773258  22.770000  32.866398  385.310608
2022-07-11  6.584469  36.437790  22.160000  32.149536  380.908966
2022-07-12  6.477166  36.289787  22

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 3, 0.8015080649513566, 1.3589989355344778, 0.5897782875276008, 76.07287920234957, 90.8411662415788, 0.8374273729604579, 44.75611551094943, 72.82247465730207, 0.614592070944703, 1.0457902985329715, 0.42311486334603804, 2.4716463285235335, 0.15502089211897974, 9.809313764237254, 11.81015581251059, 0.8305829254044363]


5 of 57 max_lookback_slice
max_lookback_slice: (103, 223, 229)
max lookback dates: 2021-10-12, 2022-04-04, 2022-04-12
df_eval dates (inclusive): 2022-04-04 - 2022-04-11
top_set_syms_n_freq: [('AMPH', 9), ('CUTR', 9), ('LNTH', 8), ('BPT', 7), ('EQT', 7), ('GLNG', 7), ('CALM', 6), ('RGLD', 5), ('SBS', 5), ('ABBV', 4), ('ADM', 4), ('CNQ', 4), ('IPI', 4), ('IRTC', 4), ('OXY', 4), ('SRE', 4), ('AR', 3), ('ATO', 3), ('CIB', 3), ('CVX', 3)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: []
sym_freq_9: ['AMPH', 'CUTR']
sym_freq_8: ['LNTH']
sym_f

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 5, -0.4867892338327658, 0.321085688242391, -1.516072661156057, -20.506839630291687, 3.9996981014646433, -5.1270968733320945, -29.071316399692243, 13.82635980379564, -2.1026008878860165, 0.8270362584342498, 0.2557573477653362, 3.233675457070646, -0.8500866306049251, -39.71628707808852, -108.82400432329452, 0.3649588831531995]

start_eval: 223,  date: 2022-04-04
end_eval:   229,  date: 2022-04-12,  df_eval last date: 2022-04-11
frequency count of symbol(s): 4

df_eval_n_SPY:
                  ABBV        ADM        CNQ         IPI        IRTC        OXY         SRE         SPY
Date                                                                                                   
2022-04-04  155.953583  88.731216  59.477077   83.139999  168.009995  57.447647  163.119492  450.903931
2022-04-05  157.437119  89.192390  58.166924   83.519997  160.259995  55.389908  161.427277  445.208405
2022-04-06  162

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 8, -0.8825273226318815, 0.11129389281110125, -7.929701265187949, -14.042748326535257, 5.118910504736835, -2.7433080366497244, -22.50692829030959, 7.430983470459959, -3.0287953646755277, 0.6159861079031296, 0.024060498700254507, 25.6015519701848, -0.45836582590859953, -24.391306482559784, -33.511114913715105, 0.7278572063437121]

start_eval: 135,  date: 2021-11-26
end_eval:   141,  date: 2021-12-06,  df_eval last date: 2021-12-03
frequency count of symbol(s): 7

df_eval_n_SPY:
             CLMT        WIRE         SPY
Date                                     
2021-11-26  16.10  138.666840  450.061676
2021-11-29  16.57  143.193130  455.582428
2021-11-30  14.78  140.345459  446.717865
2021-12-01  15.15  141.174759  441.756073
2021-12-02  15.29  146.480438  448.522125
2021-12-03  14.17  140.425385  444.619415

grp(retnStd/UI):   mean, std, mean/std:               1.1,               0.5,              

...

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 4, 1702.608008816084, 2949.6350728933553, 0.5772266625328544, 322753.10385700053, 559064.5091760573, 0.5773092345508933, 40420.984292261666, 70097.48403203179, 0.5766395877174546, 2.4988610139765943, 3.1690001075484417, 0.7885329533515633, -0.6660342078116476, -31.528358044173025, -53.19541260511006, 0.592689416251362]

start_eval: 189,  date: 2022-02-14
end_eval:   195,  date: 2022-02-23,  df_eval last date: 2022-02-22
frequency count of symbol(s): 3

df_eval_n_SPY:
                   AA        ABBV         SPY
Date                                         
2022-02-14  73.317177  137.756271  432.011353
2022-02-15  73.495850  139.451736  438.978302
2022-02-16  77.267921  140.521027  439.470306
2022-02-17  75.907997  139.654022  430.082642
2022-02-18  77.625275  138.748505  427.297821
2022-02-22  73.545479  140.222412  422.712219

grp(retnStd/UI):   mean, std, mean/std:               1.5,          

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 6, 0.21948670716862784, 1.2631268873550545, 0.17376457532958203, 20.743217356102768, 43.40859164480182, 0.4778597178604105, 16.5045371401437, 47.59069183145324, 0.34680179053912513, 0.8358778764041114, 0.2518678334104046, 3.318716269108072, 3.006699717702718, 521.9746210957633, 231.33982579283995, 2.2563111185325297]

start_eval: 137,  date: 2021-11-30
end_eval:   143,  date: 2021-12-08,  df_eval last date: 2021-12-07
frequency count of symbol(s): 5

df_eval_n_SPY:
                  BOOT         DY          HD        HPQ       PDFS       RMBS         SPY
Date                                                                                      
2021-11-30  122.339996  93.480003  389.169037  33.961613  29.930000  26.900000  446.717865
2021-12-01  121.279999  91.410004  390.593201  35.097511  30.360001  27.240000  441.756073
2021-12-02  122.199997  94.620003  397.762848  36.233418  30.190001  27.389

grp(retnStd/UI):   mean, std, mean/std:               1.2,               0.6,               2.0
grp(CAGR/retnStd): mean, std, mean/std:             610.6,           1,296.7,               0.5
grp(CAGR/UI):      mean, std, mean/std:           1,328.5,           2,690.8,               0.5
grp(CAGR):         mean, std, mean/std:              20.7,              42.3,               0.5
row_add_total: [57, '[15, 30, 60, 120]', 6, 20, 0, 10, 3, 20.738937460068193, 42.32919482187182, 0.4899440574606022, 1328.4664371305812, 2690.7767324287774, 0.4937111359408355, 610.5787274475489, 1296.6516665764566, 0.47088878469547424, 1.2153726131651663, 0.5964075482031403, 2.0378223193634066, -0.7424083304741121, -48.64517976657593, -79.49301874997744, 0.6119427910968565]
appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 3, 20.738937460068193, 42.32919482187182, 0.4899440574606022, 1328.4664371305812, 2690.7767324287774, 0.4937111359408355, 610.5787274475489, 1296.6

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 9, -0.9999139853412027, 2.0702226776969646e-05, -48299.82765204586, -10.60975142832849, 0.2999652988423831, -35.369929352739526, -46.87443622805544, 8.172290286643715, -5.7357771914519615, 0.23228903997668726, 0.03409893099250187, 6.812208864488041, -0.9896294040206132, -16.940887308136066, -49.32992542499348, 0.3434200875469559]

start_eval: 269,  date: 2022-06-09
end_eval:   275,  date: 2022-06-17,  df_eval last date: 2022-06-16
frequency count of symbol(s): 7

df_eval_n_SPY:
                   SU         SPY
Date                             
2022-06-09  40.830757  396.258514
2022-06-10  40.371437  384.768707
2022-06-13  38.670967  370.159790
2022-06-14  38.475513  369.044373
2022-06-15  37.918465  374.305573
2022-06-16  36.022545  361.917542

grp(retnStd/UI):   mean, std, mean/std:               0.3,               0.0,               inf
grp(CAGR/retnStd): mean, std, mean/std:             -55.5

appended row_add to df_eval_sym_freq_results_test:
[57, '[15, 30, 60, 120]', 6, 20, 0, 10, 4, 43.72440208507607, 83.59952848152076, 0.5230221136323887, 9180.151831021749, 14390.297893977835, 0.6379403608359998, 1964.4080656147255, 3306.528620551919, 0.5940998222137966, 2.74516789615252, 2.3086226451482217, 1.1890933764865115, -0.8680793941428455, -45.286933129205075, -55.585205411057494, 0.8147299770560207]


57 of 57 max_lookback_slice
max_lookback_slice: (86, 206, 212)
max lookback dates: 2021-09-17, 2022-03-10, 2022-03-18
df_eval dates (inclusive): 2022-03-10 - 2022-03-17
top_set_syms_n_freq: [('IPI', 10), ('OXY', 10), ('CVX', 9), ('LNTH', 9), ('ARCH', 7), ('MOS', 6), ('PTEN', 6), ('ACRS', 5), ('BPT', 5), ('FHN', 5), ('GLNG', 5), ('MRO', 5), ('RRC', 5), ('X', 5), ('AR', 4), ('ATI', 4), ('CNQ', 4), ('DVN', 4), ('HAL', 4), ('HP', 4)]

sym_freq_15: []
sym_freq_14: []
sym_freq_13: []
sym_freq_12: []
sym_freq_11: []
sym_freq_10: ['IPI', 'OXY']
sym_freq_9: ['CVX', 'LNTH']
sym_freq_8: []
s

In [14]:
if store_results:  # record results to df
  if run_type == 'train':
    pickle_dump(df_eval_sym_freq_results, path_data_dump, fp_df_eval_sym_freq_results)
    print(f'Save results to: {fp_df_eval_sym_freq_results}')
  elif run_type == 'validate':
    pickle_dump(df_eval_sym_freq_results_validate, path_data_dump, fp_df_eval_sym_freq_results_validate)   
    print(f'Save results to: {fp_df_eval_sym_freq_results_validate}')
  elif run_type == 'test':
    pickle_dump(df_eval_sym_freq_results_test, path_data_dump, fp_df_eval_sym_freq_results_test)   
    print(f'Save results to: {fp_df_eval_sym_freq_results_test}')
  else:
    msg_stop = f"ERROR: Result was not saved. run_type must be 'train', 'validate', or 'test', run_type is: {run_type}"
    raise SystemExit(msg_stop )  

Save results to: df_eval_sym_freq_results_test


In [15]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/

In [16]:
# df = pickle_load(path_data_dump, fp_df_eval_sym_freq_results)
# df.sort_values(by='sym_freq_cnt')

In [17]:
# # pd.crosstab(df['days_lookbacks'], df['grp(CAGR/UI)_mean'])
# tbl = df.groupby(['days_lookbacks', 'days_eval', 'sym_freq_cnt'])\
#         .agg({'grp(CAGR)_mean':          ['count', 'mean', 'std',],
#               'grp(CAGR/UI)_mean':       ['mean', 'std',],
#               'grp(CAGR/retnStd)_mean':  ['mean', 'std',],
#               'SPY_CAGR':                ['mean', 'std',],
#               'SPY_CAGR/UI':             ['mean', 'std',],
#               'SPY_CAGR/retnStd':        ['mean', 'std',],                           
#               })
# tbl

In [18]:
# tbl['dif_CAGR_mean'] = tbl['grp(CAGR)_mean', 'mean'] - tbl['SPY_CAGR', 'mean']
# tbl['dif_CAGR/UI_mean'] = tbl['grp(CAGR/UI)_mean', 'mean'] - tbl['SPY_CAGR/UI', 'mean'] 
# tbl['dif_CAGR/retnStd_mean'] = tbl['grp(CAGR/retnStd)_mean', 'mean'] - tbl['SPY_CAGR/retnStd', 'mean'] 
# tbl['grp_CAGR/UI_mean/std'] = tbl['grp(CAGR/UI)_mean', 'mean'] / tbl['grp(CAGR/UI)_mean', 'std']  
# tbl['SPY_CAGR/UI_mean/std'] = tbl['SPY_CAGR/UI', 'mean'] / tbl['SPY_CAGR/UI', 'std'] 
# tbl['dif_CAGR/UI_mean/std'] = tbl['grp_CAGR/UI_mean/std']  - tbl['SPY_CAGR/UI_mean/std'] 
# # tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
# # tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
# # tbl

In [19]:
# tbl.sort_values(by='dif_CAGR/UI_mean', ascending=False, inplace=True)
# tbl.head(20)

In [20]:
# tbl.sort_values(by='dif_CAGR/UI_mean/std', ascending=False, inplace=True)
# tbl.head(20)

In [21]:
# if verbose:
#   # from IPython.display import display, HTML
#   # from myUtils import symb_perf_stats_vectorized_v8
#   df = df_eval.copy()
#   df_SPY = df_SPY.copy()  
#   _my_df = [df_eval, df_SPY]
#   for _df in _my_df:
#     # print(f'_df:\n{_df}')
#     (
#       symbols,
#       period_yr,
#       retn,
#       DD,
#       UI,
#       MDD,
#       retnMean,
#       retnStd,
#       retnStd_div_UI,
#       CAGR,
#       CAGR_div_retnStd,
#       CAGR_div_UI,
#       grp_retnStd_div_UI,     
#       grp_CAGR,
#       grp_CAGR_div_retnStd,
#       grp_CAGR_div_UI,
#     ) = symb_perf_stats_vectorized_v8(_df)

#     # display(HTML(_df.to_html()))
#     print(f'_df:\n{_df}\n')
#     print(f'symbols:\n{symbols}\n')
#     print(f'period_yr:\n{period_yr}\n')
#     print(f'retn:\n{retn}\n')
#     print(f'DD:\n{DD}\n')
#     print(f'UI:\n{UI}\n')
#     print(f'MDD:\n{MDD}\n')
#     print(f'retnMean:\n{retnMean}\n')
#     print(f'retnStd:\n{retnStd}\n')
#     print(f'retnStd_div_UI:\n{retnStd_div_UI}\n')
#     print(f'CAGR:\n{CAGR}\n')
#     print(f'CAGR_div_retnStd:\n{CAGR_div_retnStd}\n')
#     print(f'CAGR_div_UI:\n{CAGR_div_UI}\n')
#     print(f'grp_retnStd_div_UI:\n{grp_retnStd_div_UI}\n')
#     print(f'grp_CAGR:\n{grp_CAGR}\n')
#     print(f'grp_CAGR_div_retnStd:\n{grp_CAGR_div_retnStd}\n')
#     print(f'grp_CAGR_div_UI:\n{grp_CAGR_div_UI}\n')            

In [22]:
# # https://stackoverflow.com/questions/31674195/plot-normal-distribution-given-mean-and-sigma-python
# # loc is mean, scale is standard deviation
# import pylab
# import numpy as np
# from scipy.stats import norm
# # x = np.linspace(-10000,100000,1000)
# x = np.linspace(-40e+10,50e+10,1000)
# y = norm.pdf(x, loc=2.562777e+10, scale=1.036925e+11)    # loc = mean, scale = standard deviation
# # z = norm.pdf(x, loc=3.540615e+10, scale=1.194430e+11)    # for example
# # z1 = norm.pdf(x, loc=298.805901, scale=826.875749)    # for example
# # z1 = norm.pdf(x, loc=1.021825, scale=1.505096)    # for example
# pylab.plot(x,y, 'b')
# # pylab.plot(x,z, 'g')
# # pylab.plot(x,z1, 'r')
# pylab.show()

In [23]:
# # get values of _cols, where grp(CAGR/retnStd)_mean is max after filtering out inf
# _cols = ['grp(CAGR/retnStd)_mean', 'grp(CAGR/retnStd)_std', 'grp(CAGR/retnStd)_mean/std']
# # _df_no_inf = df.loc[df['grp(CAGR/retnStd)_mean'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# # _idx = _df_no_inf['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# _idx = df['grp(CAGR/retnStd)_mean'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# grp_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# grp_inf_replacement


In [24]:
# # get values of _cols, where SPY_CAGR/retnStd is max after filtering out inf
# _cols = ['SPY_CAGR/retnStd']
# # _df_no_inf = df.loc[df['SPY_CAGR/retnStd'] != np.inf]  # df with filter out inf in column grp(CAGR/UI)_mean 
# _idx = df['SPY_CAGR/retnStd'].idxmax()  # index value of max in grp(CAGR/UI)_mean 
# SPY_inf_replacement = df.loc[[_idx], _cols].squeeze()  # convert df (only has 1 row) to series
# print(f'_idx: {_idx}')
# SPY_inf_replacement